In [3]:
%load_ext autoreload
%autoreload 2
import pandas as pd
import numpy as np
import nltk
from sklearn import datasets
from sklearn.feature_extraction.text import CountVectorizer
import plotly
plotly.tools.set_credentials_file(username='coolcoolkg21', api_key='I5SSdlMkMxR3qwdHnC98')
import plotly.graph_objs as go
import plotly.plotly as py
import math
from io import StringIO
%matplotlib inline

# my functions
import helpers.data_mining_helpers as dmh
import helpers.text_analysis as ta

In [4]:
'''
open txt file and convert to dataframe
'''
amazon = open("sentiment labelled sentences/amazon_cells_labelled.txt", 'r')
a_lines = amazon.read().splitlines()
a_good = 0
a_bad = 0
a_dict = {}
a_dict['comment'] = []
a_dict['talk'] = []
a_dict['type'] = []
# A = pd.read_table(open("sentiment labelled sentences/amazon_cells_labelled.txt"), header=None)
for i in range(len(a_lines)):
    a_comment = a_lines[i].split('\t')
    if a_comment[1] == '0':
        a_bad += 1
    elif a_comment[1] == '1':
        a_good += 1
    a_dict['comment'].append(a_comment[0])
    a_dict['talk'].append(a_comment[1])
    a_dict['type'].append('amazon')
# print('good:', a_dict['good'], 'bad: ', a_dict['bad'])
A = pd.DataFrame(a_dict, columns = a_dict.keys())
A

,comment,talk,type
0,So there is no way for me to plug it in here i...,0,amazon
1,"Good case, Excellent value.",1,amazon
2,Great for the jawbone.,1,amazon
3,Tied to charger for conversations lasting more...,0,amazon
4,The mic is great.,1,amazon
5,I have to jiggle the plug to get it to line up...,0,amazon
6,If you have several dozen or several hundred c...,0,amazon
7,If you are Razr owner...you must have this!,1,amazon
8,"Needless to say, I wasted my money.",0,amazon
9,What a waste of money and time!.,0,amazon


In [5]:
'''
Check the missing values
'''
A.isnull().apply(lambda x: dmh.check_missing_values(x))


comment    (The amoung of missing records is: , 0)
talk       (The amoung of missing records is: , 0)
type       (The amoung of missing records is: , 0)
dtype: object

In [6]:
'''
Check duplicated data and delete the duplicates
'''
A.duplicated()
print(sum(A.duplicated()))
print(len(A))
A.drop_duplicates(keep='last', inplace=True)
print(len(A))

10
1000
990


In [7]:
"""
sampling A
"""

A_sample = A.sample(n=100)
A_talk_counts = ta.get_tokens_and_frequency(list(A.talk))
A_sample_talk_counts = ta.get_tokens_and_frequency(list(A_sample.talk))
# py.iplot(ta.plot_word_frequency(A_sample_talk_counts, "Talk distribution"))
ori_count = list(A_talk_counts[1])
sample_count = list(A_sample_talk_counts[1])


In [8]:
"""
Bar charts
"""

trace1 = go.Bar(
    x=['Good Talk', 'Bad Talk'],
    y=ori_count,
    name='Original'
)
trace2 = go.Bar(
    x=['Good Talk', 'Bad Talk'],
    y=sample_count,
    name='Sample'
)

data = [trace1, trace2]
layout = go.Layout(
    barmode='stack'
)

fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='stacked-bar')

In [9]:
"""
open txt file and convert to dataframe
"""

imdb = open("sentiment labelled sentences/imdb_labelled.txt", 'r')
i_lines = imdb.read().splitlines()
i_dict = {}
i_dict['comment'] = []
i_dict['talk'] = []
i_dict['type'] = []
# I = pd.read_table(open("sentiment labelled sentences/imdb_labelled.txt"), header=None)
for i in range(len(i_lines)):
    i_comment = i_lines[i].split('\t')
#     i_dict[i] = i_comment
    i_dict['comment'].append(i_comment[0])
    if len(i_comment) > 1:                 # list index will out of range(There are some missing values)
        i_dict['talk'].append(i_comment[1])
    else:
        i_dict['talk'].append(None)
    i_dict['type'].append('imdb')
I = pd.DataFrame(data = i_dict, columns = i_dict.keys())
I.isnull().apply(lambda x: dmh.check_missing_values(x))


comment    (The amoung of missing records is: , 0)
talk       (The amoung of missing records is: , 2)
type       (The amoung of missing records is: , 0)
dtype: object

In [10]:
I.isnull().any(axis=1) # row

0       False
1       False
2       False
3       False
4       False
5       False
6       False
7       False
8       False
9       False
10      False
11      False
12      False
13      False
14      False
15      False
16      False
17      False
18      False
19      False
20      False
21      False
22      False
23      False
24      False
25      False
26      False
27      False
28      False
29      False
        ...  
972     False
973     False
974     False
975     False
976     False
977     False
978     False
979     False
980     False
981     False
982     False
983     False
984     False
985     False
986     False
987     False
988     False
989     False
990     False
991     False
992     False
993     False
994     False
995     False
996     False
997     False
998     False
999     False
1000    False
1001    False
Length: 1002, dtype: bool

In [11]:
'''
delete the whole missing value row
'''
print(len(I))
I.dropna(inplace=True)
print(len(I))

1002
1000


In [12]:
I.isnull().apply(lambda x: dmh.check_missing_values(x))

comment    (The amoung of missing records is: , 0)
talk       (The amoung of missing records is: , 0)
type       (The amoung of missing records is: , 0)
dtype: object

In [13]:
'''
Check duplicated data and delete the duplicates
'''
I.duplicated()
print(sum(I.duplicated()))
print(len(I))
I.drop_duplicates(keep='last', inplace=True)
print(len(I))

3
1000
997


In [14]:
"""
sampling I
"""

I_sample = I.sample(n=500)
I_talk_counts = ta.get_tokens_and_frequency(list(I.talk))
I_sample_talk_counts = ta.get_tokens_and_frequency(list(I_sample.talk))
# plotly.plotly.iplot(ta.plot_word_frequency(I_sample_talk_counts, "Talk distribution"))

In [15]:
"""
Bar charts
"""
ori_count = list(I_talk_counts[1])
sample_count = list(I_sample_talk_counts[1])


trace1 = go.Bar(
    x=['Good Talk', 'Bad Talk'],
    y=ori_count,
    name='Original'
)
trace2 = go.Bar(
    x=['Good Talk', 'Bad Talk'],
    y=sample_count,
    name='Sample'
)

data = [trace1, trace2]
layout = go.Layout(
    barmode='group'
)

fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='grouped-bar')

In [16]:
'''
open txt file and convert to dataframe
'''
yelp = open("sentiment labelled sentences/yelp_labelled.txt", 'r')
y_lines = yelp.read().splitlines()
y_dict = {}
y_dict['comment'] = []
y_dict['talk'] = []
y_dict['type'] = []
for i in range(len(y_lines)):
    y_comment = y_lines[i].split('\t')
    y_dict['comment'].append(y_comment[0])
    if len(y_comment) > 1:                 # list index will out of range(There are some missing values)
        y_dict['talk'].append(y_comment[1])
    else:
        y_dict['talk'].append(None)
    y_dict['type'].append('yelp')
Y = pd.DataFrame(data = y_dict, columns = y_dict.keys())
Y.isnull().apply(lambda x: dmh.check_missing_values(x))

comment    (The amoung of missing records is: , 0)
talk       (The amoung of missing records is: , 0)
type       (The amoung of missing records is: , 0)
dtype: object

In [17]:
'''
Check duplicated data and delete the duplicates
'''
Y.duplicated()
print(sum(Y.duplicated()))
print(len(Y))
Y.drop_duplicates(keep='last', inplace=True)
print(len(Y))

4
1000
996


In [18]:
"""
sampling Y
"""

Y_sample = Y.sample(n=500)
Y_talk_counts = ta.get_tokens_and_frequency(list(Y.talk))
Y_sample_talk_counts = ta.get_tokens_and_frequency(list(Y_sample.talk))
# plotly.plotly.iplot(ta.plot_word_frequency(Y_sample_talk_counts, "Talk distribution"))

In [19]:
"""
Bar charts
"""
ori_count = list(Y_talk_counts[1])
sample_count = list(Y_sample_talk_counts[1])


trace1 = go.Bar(
    x=['Good Talk', 'Bad Talk'],
    y=ori_count,
    name='Original'
)
trace2 = go.Bar(
    x=['Good Talk', 'Bad Talk'],
    y=sample_count,
    name='Sample'
)

data = [trace1, trace2]
layout = go.Layout(
    barmode='group'
)

fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='grouped-bar')

In [20]:
"""
merge three dataset to one data
and do the operations
"""

combine = {}
combine['comment'] = []
combine['talk'] = []
combine['type'] = []

for i in range(len(a_lines)):
    a_comment = a_lines[i].split('\t')
    combine['comment'].append(a_comment[0])
    if len(a_comment) > 1:                 # list index will out of range(There are some missing values)
        combine['talk'].append(a_comment[1])
    else:
        combine['talk'].append(None)
    combine['type'].append('amazon')

for i in range(len(i_lines)):
    i_comment = i_lines[i].split('\t')
    combine['comment'].append(i_comment[0])
    if len(i_comment) > 1:                 # list index will out of range(There are some missing values)
        combine['talk'].append(i_comment[1])
    else:
        combine['talk'].append(None)
    combine['type'].append('imdb')
for i in range(len(y_lines)):
    y_comment = y_lines[i].split('\t')
    combine['comment'].append(y_comment[0])
    if len(y_comment) > 1:                 # list index will out of range(There are some missing values)
        combine['talk'].append(y_comment[1])
    else:
        combine['talk'].append(None)
    combine['type'].append('yelp')
myDF = pd.DataFrame(data = combine, columns = combine.keys())
myDF

,comment,talk,type
0,So there is no way for me to plug it in here i...,0,amazon
1,"Good case, Excellent value.",1,amazon
2,Great for the jawbone.,1,amazon
3,Tied to charger for conversations lasting more...,0,amazon
4,The mic is great.,1,amazon
5,I have to jiggle the plug to get it to line up...,0,amazon
6,If you have several dozen or several hundred c...,0,amazon
7,If you are Razr owner...you must have this!,1,amazon
8,"Needless to say, I wasted my money.",0,amazon
9,What a waste of money and time!.,0,amazon


In [21]:
"""
check missing values & delete them
"""
myDF.isnull().apply(lambda x: dmh.check_missing_values(x))
myDF.dropna(inplace=True)


In [22]:
"""
find & delete the duplicates
"""

myDF.duplicated()
print(sum(myDF.duplicated()))
I.drop_duplicates(keep='last', inplace=True)

17


In [23]:
"""
sampling
"""
myDF_sample = myDF.sample(n=1000)
type_counts = ta.get_tokens_and_frequency(list(myDF.type))
myDF_sample_counts = ta.get_tokens_and_frequency(list(myDF_sample.type))

types = ['amazon', 'imdb', 'yelp']
ori_count = list(type_counts[1])
sample_count = list(myDF_sample_counts[1])

trace1 = go.Bar(
    x = types,
    y = ori_count,
    name = 'Original'
)
trace2 = go.Bar(
    x = types,
    y = sample_count,
    name = 'Sample'
)

data = [trace1, trace2]
layout = go.Layout(
    barmode='group'
)

fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='grouped-bar')

In [24]:
count_vect = CountVectorizer()
DF_counts = count_vect.fit_transform(myDF.comment)

In [25]:
analyze = count_vect.build_analyzer()
analyze(" ".join(list(myDF[0:1].comment)))

['so',
 'there',
 'is',
 'no',
 'way',
 'for',
 'me',
 'to',
 'plug',
 'it',
 'in',
 'here',
 'in',
 'the',
 'us',
 'unless',
 'go',
 'by',
 'converter']

In [26]:
"""
shape of the matrix
"""

DF_counts.shape

(3000, 5155)

In [27]:
count_vect.get_feature_names()[0:-1]
DF_counts.toarray()

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ..., 
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [28]:
count_vect.transform(['00 Great']).toarray()

array([[1, 0, 0, ..., 0, 0, 0]])

In [29]:
from  plotly.offline import plot
plot_x = ["term_"+str(i) for i in count_vect.get_feature_names()[0:-1]]
plot_y = ["doc_"+ str(i) for i in list(myDF.index)[0:-1]]
plot_z = DF_counts[0:-1, 0:-1].toarray()
plot(ta.plot_heat_map(plot_x, plot_y, plot_z))

'file:///Users/coolcoolkg21/data_mining_hw_1/temp-plot.html'

In [30]:
from sklearn.decomposition import PCA

In [31]:
DF_reduced = PCA(n_components=3).fit_transform(DF_counts.toarray())
trace1 = ta.get_trace(DF_reduced, myDF["type"], "amazon", "rgb(71,233,163)")
trace2 = ta.get_trace(DF_reduced, myDF["type"], "imdb", "rgb(52,133,252)")
trace3 = ta.get_trace(DF_reduced, myDF["type"], "yelp", "rgb(229,65,136)")
data = [trace1, trace2, trace3]
layout = go.Layout(
    margin=dict(
        l=0,
        r=0,
        b=0,
        t=0
    )
)
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='simple-3d-scatter')

In [32]:
term_frequencies = []
for i in range(0,DF_counts.shape[1]):
    term_frequencies.append(sum(DF_counts[:,i].toarray()))
py.iplot(ta.plot_word_frequency([count_vect.get_feature_names(), term_frequencies], "Term Frequency Distribution"))

PlotlyRequestError: Account limit reached: Your account is limited to creating 25 public charts. To continue, you can override or delete existing charts or you can upgrade your account at: https://plot.ly/products/cloud

In [ ]:
term_frequencies_log = [math.log(i) for i in term_frequencies]
py.iplot(ta.plot_word_frequency([count_vect.get_feature_names(), term_frequencies_log], "Term Frequency Distribution"))

In [33]:
from sklearn import preprocessing, metrics, decomposition, pipeline, dummy
mlb = preprocessing.LabelBinarizer()
mlb.fit(myDF.type)
mlb.classes_
myDF['bin_category'] = mlb.transform(myDF['type']).tolist()
myDF[0:9]

,comment,talk,type,bin_category
0,So there is no way for me to plug it in here i...,0,amazon,"[1, 0, 0]"
1,"Good case, Excellent value.",1,amazon,"[1, 0, 0]"
2,Great for the jawbone.,1,amazon,"[1, 0, 0]"
3,Tied to charger for conversations lasting more...,0,amazon,"[1, 0, 0]"
4,The mic is great.,1,amazon,"[1, 0, 0]"
5,I have to jiggle the plug to get it to line up...,0,amazon,"[1, 0, 0]"
6,If you have several dozen or several hundred c...,0,amazon,"[1, 0, 0]"
7,If you are Razr owner...you must have this!,1,amazon,"[1, 0, 0]"
8,"Needless to say, I wasted my money.",0,amazon,"[1, 0, 0]"
